In [14]:
import kglab

ttl_text = """
@prefix : <http://www.w3.org/2012/12/rdf-val/SOTA-ex#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:peep0 a foaf:Person ;
    foaf:givenName "Alice" ;
    foaf:familyName "Nakamoto" ;
    foaf:phone "+1.555.222.2222" ;
    foaf:mbox "alice@example.org" .

:peep1 a foaf:Person ;
    foaf:givenName "Bob" ;
    foaf:familyName "Patel" ;
    foaf:phone "+1.555.666.5150" ;
    foaf:mbox "bob@example.org" .

:peep2 a foaf:Person ;
    foaf:givenName "Dhanya" ;
    foaf:familyName "O'Neill" ;
    foaf:phone "+1.555.123.9876" ;
    foaf:mbox "dhanya@example.org" .
"""

kg = kglab.KnowledgeGraph(
    name = "A KG example about people"
    )

kg.load_rdf_text(ttl_text)

In [15]:
sparql = """
    SELECT ?person ?surname ?email
    WHERE {
        ?person foaf:familyName ?surname .
        ?person foaf:mbox ?email .
    }
    ORDER BY DESC(?surname)
"""

In [16]:
import pandas as pd
import great_expectations as ge

pd.set_option("max_rows", None)

df = kg.query_as_df(sparql, bindings=None, simplify=False, pythonify=True)

In [17]:
my_df = ge.from_pandas(df)
my_df.head()

,person,surname,email
0,http://www.w3.org/2012/12/rdf-val/SOTA-ex#peep1,Patel,bob@example.org
1,http://www.w3.org/2012/12/rdf-val/SOTA-ex#peep2,O'Neill,dhanya@example.org
2,http://www.w3.org/2012/12/rdf-val/SOTA-ex#peep0,Nakamoto,alice@example.org


In [18]:
my_df.expect_table_columns_to_match_ordered_list(column_list=["person", "surname", "email"])

{
  "meta": {},
  "result": {
    "observed_value": [
      "person",
      "surname",
      "email"
    ]
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true
}

In [19]:
my_df.expect_column_values_to_be_unique(column="person")

{
  "meta": {},
  "result": {
    "element_count": 3,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true
}

In [23]:
my_df.expect_column_values_to_match_regex(column="email", regex="(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|\"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*\")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])")

{
  "meta": {},
  "result": {
    "element_count": 3,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true
}

In [24]:
import json

with open( "rdf_test_expectation.json", "w") as my_file:
    my_file.write(
        json.dumps(my_df.get_expectation_suite().to_json_dict())
    )